In [53]:
import numpy as np
import matplotlib.pyplot as plt



In [54]:
# Consts
# Solar energy flux at Earth
We = 1361 # W/m^2
c = 3e8 # m/s
sigma = 20 # kg/m^2 
Re = 6.371e3 # km
mu_s = 1.327e11 # km^3/s^2

D = 1.496e8 # km 1 AU
# T such that mu_s = D^3/T^2
T = np.sqrt(D**3/mu_s)


# Uknwowns are [l_r, l_vr, l_vtheta]

In [55]:
beta = 2*We/ (c * sigma) #[m/s^2]
beta = beta*1e-3*D**2 # [km^3/s^2]
# Adimensionalize beta
beta_adim = beta*T**2/D**3

print(f"beta = {beta_adim}")

beta = 7.651192604873147e-05


In [45]:

# Adimensionalization parameters 
# mu_s = 1, D = 1 AU, T such that 



mu_s_adim = mu_s * T**2 / D**3


print(f"mu_s = {mu_s_adim}, D = {D}, T = {T}")


mu_s = 1.0, D = 149600000.0, T = 5022985.550771121


In [46]:
# Velocity of Earth
v_earth = np.sqrt(mu_s/D)
v_merc = np.sqrt(mu_s/(0.387*D))
print("v_earth = ", v_earth)
print("v_merc = ", v_merc)

v_earth =  29.783083882658914
v_merc =  47.87559474837184


In [47]:
### Adimensionalized initial and final conditions

# Initial conditions (position and angular velocity)
r0 = 1 # AU
vr0 = 0 # radial velocity
vt0 = 1 # angular velocity


# Final conditions
rf = 0.387 # AU
vrf = 0
vtf = v_merc*T/D 

# Adimensionalize beta
beta_adim = beta*T**2/D**3

In [48]:
X0_adim = np.array([r0, 0, vr0, vt0])
Xf_adim = np.array([rf, 0, vrf, vtf])

In [49]:
# Print adimensionalized parameters
print(f"r0 = {r0}, vr0 = {vr0}, vt0 = {vt0}")
print(f"rf = {rf}, vrf = {vrf}, vtf = {vtf}")
print(f"beta = {beta_adim}")

r0 = 1, vr0 = 0, vt0 = 1
rf = 0.387, vrf = 0, vtf = 1.6074760739013738
beta = 7.651192604873147e-05


In [ ]:
def f_dyn(t, X):


    # Split into state and costate
    S, L = X[:4], X[4:]

    # Unpack state
    r, theta, vr, vtheta = S
    # Unpack costate
    p_r, p_theta, p_vr, p_vtheta = L

    # Compute optimal control (thrust angle)
    # u = alpha = atan(-3*p_vr + sqrt(9*p_vr**2 + 8*p_vtheta**2)/(4*p_vtheta)
    # T = 2*We/(c*sigma)(Rf^2/r^2)*cos(alpha)
    # beta = 2*We/c*sigma*(Re^2)  ---> T_om = beta/r^2 * cos^2(alpha)
    alpha = np.atan(-3*p_vr + np.sqrt(9*p_vr**2 + 8*p_vtheta**2)/(4*p_vtheta))
    # Dynamics
    dS = np.zeros(4)
    dL = np.zeros(4)

    dS[0] = vr
    dS[1] = vtheta/r
    dS[2] = vtheta**2/r - 1/r**2 + beta/r**2 * np.cos(alpha)**3
    dS[3] = -vr*vtheta/r + beta/r**2 * np.cos(alpha)**2 * np.sin(alpha)

    dL[0] = p_theta*vtheta/r**2 + p_vr*(vtheta**2/r**2 - 2/r**3 + 2*beta/r**3*np.cos(alpha)**3) - p_vtheta*(vr*vtheta/r**2 - 2*beta/r**3*np.cos(alpha)**2*np.sin(alpha))
    dL[1] = 0
    dL[2] = -p_r - p_vtheta*vtheta/r
    dL[3] = -p_theta/r - 2*p_vr*vtheta/r + p_vtheta*vr/r
    dL[4] = 0

    return np.concatenate((dS, dL))
